### Question 1:

- "Remove any object that has `{attribute}` = `{value}`. How many objects remain that have `{color}` and `{shape}`?"
- Question Template: Delete,  Question Type: Existence

In [2]:
import json

# --- Load JSON from file ---
json_file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000000.json"  # Change path as needed

with open(json_file_path, "r") as f:
    scene_data = json.load(f)


def count_after_removal(data, remove_attribute, remove_value, target_color, target_shape):
    """
    Removes objects based on a condition and counts remaining objects matching color and shape.

    Args:
        data (dict): The dictionary loaded from the JSON scene file.
        remove_attribute (str): The attribute key to check for removal (e.g., "size").
        remove_value (str): The value of the attribute that triggers removal (e.g., "large").
        target_color (str): The color to count in the remaining objects.
        target_shape (str): The shape to count in the remaining objects.

    Returns:
        int: The count of remaining objects matching the target color and shape.
    """
    if 'objects' not in data:
        print("Error: 'objects' key not found in data.")
        return 0

    remaining_objects = []
    for obj in data['objects']:
        # Check if the object has the removal attribute and if its value matches the removal value
        if remove_attribute in obj and obj[remove_attribute] == remove_value:
            continue # Skip this object (simulate removal)
        remaining_objects.append(obj)

    count = 0
    for obj in remaining_objects:
        # Check if the remaining object has the target color and shape
        if obj.get('color') == target_color and obj.get('shape') == target_shape:
            count += 1

    print(f"Removed objects where {remove_attribute} = {remove_value}.")
    print(f"Found {count} remaining objects with color = {target_color} and shape = {target_shape}.")
    return count

# --- Example Usage for Question 1 ---
# Remove any object that has material = metal. How many objects remain that have color=blue and shape=cylinder?
remove_attr = "material"
remove_val = "metal"
target_col = "blue"
target_shp = "cylinder"

count1 = count_after_removal(scene_data, remove_attr, remove_val, target_col, target_shp)

print("-" * 20)

# Remove any object that has size = large. How many objects remain that have color=gray and shape=cube?
remove_attr = "size"
remove_val = "large"
target_col = "gray"
target_shp = "cube"

count2 = count_after_removal(scene_data, remove_attr, remove_val, target_col, target_shp)

Removed objects where material = metal.
Found 1 remaining objects with color = blue and shape = cylinder.
--------------------
Removed objects where size = large.
Found 1 remaining objects with color = gray and shape = cube.


### Question 2:

In [3]:
import json
import copy  # Needed for deepcopy

# --- Load JSON from file ---
# json_file_path = "/content/output_scenes/CLEVR_new_000000.json"  # Change path as needed

# with open(json_file_path, "r") as f:
#     scene_data = json.load(f)
def load_scene(file_path):
    json_file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/"+file_path
    with open(json_file_path, "r") as f:
        scene_data = json.load(f)
    return scene_data

def count_after_color_change(data, original_color, new_color):
    """
    Changes the color of objects and counts how many objects still have the original color.

    Args:
        data (dict): The dictionary loaded from the JSON scene file.
        original_color (str): The color of objects to change.
        new_color (str): The color to change the objects to.

    Returns:
        tuple: A tuple containing:
            - int: The count of objects that *still* have the original_color.
            - int: The count of objects that *now* have the new_color.
    """
    if 'objects' not in data:
        print("Error: 'objects' key not found in data.")
        return 0, 0

    # Create a deep copy to avoid modifying the original data
    modified_data = copy.deepcopy(data)
    objects_to_change = 0

    for obj in modified_data['objects']:
        if obj.get('color') == original_color:
            obj['color'] = new_color
            objects_to_change += 1

    print(f"Changed {objects_to_change} objects from {original_color} to {new_color}.")

    # Count how many objects *still* have the original color
    original_color_remaining_count = sum(1 for obj in modified_data['objects'] if obj.get('color') == original_color)

    # Count how many objects *now* have the new color
    new_color_count = sum(1 for obj in modified_data['objects'] if obj.get('color') == new_color)

    print(f"Found {original_color_remaining_count} objects remaining with color={original_color}.")
    print(f"Found {new_color_count} objects now with color={new_color}.")

    return original_color_remaining_count, new_color_count

In [6]:
scene_data = load_scene("CLEVR_new_000000.json")
# --- Example Usage ---
# scene_data = load_scene()
# Change all gray objects to yellow. How many gray objects remain?
orig_color = "gray"
new_col = "yellow"
remaining_count, new_count = count_after_color_change(scene_data, orig_color, new_col)

print("-" * 20)

# Change all purple objects to red. How many purple objects remain?
orig_color = "purple"
new_col = "red"
remaining_count2, new_count2 = count_after_color_change(scene_data, orig_color, new_col)


Changed 2 objects from gray to yellow.
Found 0 objects remaining with color=gray.
Found 2 objects now with color=yellow.
--------------------
Changed 1 objects from purple to red.
Found 0 objects remaining with color=purple.
Found 2 objects now with color=red.


In [7]:
scene_data = load_scene("CLEVR_new_000001.json")
# --- Example Usage ---
# scene_data = load_scene()
# Change all gray objects to yellow. How many gray objects remain?
orig_color = "yellow"
new_col = "red"
remaining_count, new_count = count_after_color_change(scene_data, orig_color, new_col)

Changed 2 objects from yellow to red.
Found 0 objects remaining with color=yellow.
Found 3 objects now with color=red.


### Questions 4

In [1]:
# # Script using 3D coordinates for viewpoint change
# import json
# import math

# # --- Vector Math Helpers ---
# def subtract_vectors(v1, v2):
#     """Subtracts v2 from v1."""
#     if v1 is None or v2 is None or len(v1) != len(v2):
#         return None
#     return [a - b for a, b in zip(v1, v2)]

# def dot_product(v1, v2):
#     """Calculates the dot product of v1 and v2."""
#     if v1 is None or v2 is None or len(v1) != len(v2):
#         return 0 # Or raise an error? Returning 0 might hide issues. Let's check inputs later.
#     return sum(a * b for a, b in zip(v1, v2))

# def vector_magnitude(v):
#     """Calculates the magnitude (length) of a vector."""
#     if v is None: return 0
#     return math.sqrt(sum(a * a for a in v))

# def normalize_vector(v):
#     """Normalizes a vector to unit length."""
#     if v is None: return None
#     mag = vector_magnitude(v)
#     if mag == 0:
#         return [0.0] * len(v) # Or handle as error?
#     return [a / mag for a in v]

# # --- Object Identification Helpers ---
# def find_object_index_by_attrs(objects, attrs):
#     """Finds the index of the first object matching all given attributes."""
#     for i, obj in enumerate(objects):
#         if not isinstance(obj, dict): continue # Skip if not a dictionary
#         match = True
#         for key, value in attrs.items():
#             if obj.get(key) != value:
#                 match = False
#                 break
#         if match:
#             return i
#     return -1 # Not found

# def get_object_description(objects, index):
#     """Generates a string description for an object at a given index."""
#     if 0 <= index < len(objects) and isinstance(objects[index], dict):
#         obj = objects[index]
#         # Filter out None or empty values before joining
#         parts = [
#             obj.get('size'), obj.get('color'), obj.get('material'), obj.get('shape')
#         ]
#         desc = ' '.join(filter(None, parts))
#         return f"the {desc}" if desc else "unknown object"
#     return "unknown object"

# # --- Main Logic ---
# def find_objects_left_from_new_view(data, ref_object_attrs):
#     """
#     Finds objects to the 'left' of a reference object when viewing from the 'right',
#     using 3D coordinate calculations.
#     """
#     if 'objects' not in data or 'directions' not in data:
#         print("Error: 'objects' or 'directions' key not found in data.")
#         return []
    
#     objects = data['objects']
#     directions = data['directions']

#     # 1. Identify Reference Object
#     ref_index = find_object_index_by_attrs(objects, ref_object_attrs)
#     if ref_index == -1:
#         print(f"Error: Reference object with attributes {ref_object_attrs} not found.")
#         return []
#     ref_obj = objects[ref_index]
#     ref_coords = ref_obj.get('3d_coords')
#     if ref_coords is None or len(ref_coords) != 3:
#         print(f"Error: Missing or invalid 3D coordinates for reference object {ref_object_attrs}.")
#         return []
#     ref_desc = get_object_description(objects, ref_index)
#     print(f"Reference object: {ref_desc} at {ref_coords}")

#     # 2. Define New Coordinate System Vectors (Based on Original Directions)
#     # New 'left' is original 'front'
#     new_left_vec = directions.get('front')
#     # New 'front' is original 'left'
#     new_front_vec = directions.get('left')

#     if new_left_vec is None or new_front_vec is None or len(new_left_vec)!=3 or len(new_front_vec)!=3:
#         print("Error: Invalid 'front' or 'left' direction vectors in JSON.")
#         return []
        
#     # Normalization is good practice but might not be strictly necessary
#     # if only comparing relative projections. Let's use original vectors for now.
#     # new_left_unit_vec = normalize_vector(new_left_vec)
#     # new_front_unit_vec = normalize_vector(new_front_vec)
    
#     print(f"Using new 'left' direction vector: {new_left_vec} (original 'front')")
#     print(f"Using new 'front' direction vector: {new_front_vec} (original 'left')")


#     objects_to_left = []
#     tolerance = 0.1 # Minimum projection value onto 'left' to be considered

#     # 3. Iterate through Candidate Objects
#     for i, cand_obj in enumerate(objects):
#         if i == ref_index:
#             continue # Skip self-comparison

#         if not isinstance(cand_obj, dict): continue

#         cand_coords = cand_obj.get('3d_coords')
#         if cand_coords is None or len(cand_coords) != 3:
#             continue # Skip objects without valid coordinates

#         # 4. Calculate Relative Position Vector
#         relative_vec = subtract_vectors(cand_coords, ref_coords)
#         if relative_vec is None: continue

#         # 5. Calculate Projections onto New Axes
#         proj_left = dot_product(relative_vec, new_left_vec)
#         proj_front = dot_product(relative_vec, new_front_vec)
        
#         cand_desc = get_object_description(objects, i)
#         # print(f"  Checking {cand_desc}: Proj_Left={proj_left:.2f}, Proj_Front={proj_front:.2f}")


#         # 6. Apply Condition
#         if proj_left > tolerance and proj_left > abs(proj_front):
#             objects_to_left.append(cand_desc)
#             # print(f"    -> Qualifies as 'left'")


#     return objects_to_left

# # --- Main Execution ---
# if __name__ == "__main__":
#     json_file_path ="/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000000.json"
#     try:
#         with open(json_file_path, 'r') as f:
#             scene_data = json.load(f)
#     except FileNotFoundError:
#         print(f"Error: JSON file not found at {json_file_path}")
#         exit()
#     except json.JSONDecodeError:
#         print(f"Error: Could not decode JSON from {json_file_path}")
#         exit()
#     except Exception as e:
#         print(f"An unexpected error occurred loading JSON: {e}")
#         exit()


#     print("\n--- Running Question with 3D Coords ---")
#     print("Query: If looking from the right side, what object(s) are to the left of the purple cube?")

#     # Attributes to uniquely identify the reference object
#     ref_attrs = {"color": "purple", "shape": "cube"}

#     left_objects = find_objects_left_from_new_view(scene_data, ref_attrs)

#     print("\n--- Results ---")
#     if left_objects:
#         print(f"Objects found to the 'left' of the '{get_object_description(scene_data['objects'], find_object_index_by_attrs(scene_data['objects'], ref_attrs))}' from the new viewpoint:")
#         for desc in left_objects:
#             print(f"- {desc}")
#     else:
#         print(f"No objects found primarily to the 'left' of the reference object based on this calculation.")


In [2]:
# Script using 3D coordinates for viewpoint change (extended)
import json
import math

# --- Vector Math Helpers ---
def subtract_vectors(v1, v2):
    """Subtracts v2 from v1."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3: # Ensure 3D
        return None
    return [a - b for a, b in zip(v1, v2)]

def dot_product(v1, v2):
    """Calculates the dot product of v1 and v2."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3:
        # Return None or raise error to indicate problem more clearly
        return None 
    return sum(a * b for a, b in zip(v1, v2))

def vector_magnitude(v):
    """Calculates the magnitude (length) of a vector."""
    if v is None or len(v)!=3: return 0
    return math.sqrt(sum(a * a for a in v))

def normalize_vector(v):
    """Normalizes a vector to unit length."""
    if v is None or len(v)!=3: return None
    mag = vector_magnitude(v)
    if mag == 0:
        return [0.0] * 3
    return [a / mag for a in v]

# --- Object Identification Helpers ---
def find_object_index_by_attrs(objects, attrs):
    """Finds the index of the first object matching all given attributes."""
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            return i
    return -1

def get_object_description(objects, index):
    """Generates a string description for an object at a given index."""
    if 0 <= index < len(objects) and isinstance(objects[index], dict):
        obj = objects[index]
        parts = [
            obj.get('size'), obj.get('color'), obj.get('material'), obj.get('shape')
        ]
        desc = ' '.join(filter(None, parts))
        return f"the {desc}" if desc else "unknown object"
    return "unknown object"

# --- Core Logic for Relative Direction Calculation ---
def find_objects_relative_direction_from_new_view(
    data,
    ref_object_attrs,
    primary_direction_original_key, # e.g., 'front' if looking for new 'left'
    orthogonal_direction_original_key # e.g., 'left' if primary is new 'left'/'right'
    ):
    """
    General function to find objects in a specific relative direction
    from a reference object when viewing from the 'right', using 3D coordinates.

    Args:
        data (dict): The loaded scene data.
        ref_object_attrs (dict): Attributes to find the reference object.
        primary_direction_original_key (str): The key in `data['directions']`
            that corresponds to the *primary* direction vector in the *new* view.
        orthogonal_direction_original_key (str): The key in `data['directions']`
            that corresponds to an *orthogonal* direction vector in the *new* view,
            used for comparison.

    Returns:
        list: A list of descriptions of objects found in the specified direction.
    """
    if 'objects' not in data or 'directions' not in data:
        print("Error: 'objects' or 'directions' key not found in data.")
        return []

    objects = data['objects']
    directions = data['directions']

    # 1. Identify Reference Object
    ref_index = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_index == -1:
        print(f"Error: Reference object with attributes {ref_object_attrs} not found.")
        return []
    ref_obj = objects[ref_index]
    ref_coords = ref_obj.get('3d_coords')
    if ref_coords is None or len(ref_coords) != 3:
        print(f"Error: Missing or invalid 3D coordinates for reference object {ref_object_attrs}.")
        return []

    # 2. Define Primary and Orthogonal Direction Vectors (from Original JSON)
    primary_vec = directions.get(primary_direction_original_key)
    orthogonal_vec = directions.get(orthogonal_direction_original_key)

    if primary_vec is None or orthogonal_vec is None or len(primary_vec)!=3 or len(orthogonal_vec)!=3:
        print(f"Error: Invalid direction vectors for keys '{primary_direction_original_key}' or '{orthogonal_direction_original_key}'.")
        return []

    # --- Optional: Normalize direction vectors ---
    # primary_vec = normalize_vector(primary_vec)
    # orthogonal_vec = normalize_vector(orthogonal_vec)
    # if primary_vec is None or orthogonal_vec is None:
    #     print("Error normalizing direction vectors.")
    #     return []
    # ---------------------------------------------

    found_objects = []
    tolerance = 0.1 # Minimum projection value onto primary axis to be considered

    # 3. Iterate through Candidate Objects
    for i, cand_obj in enumerate(objects):
        if i == ref_index: continue
        if not isinstance(cand_obj, dict): continue

        cand_coords = cand_obj.get('3d_coords')
        if cand_coords is None or len(cand_coords) != 3: continue

        # 4. Calculate Relative Position Vector
        relative_vec = subtract_vectors(cand_coords, ref_coords)
        if relative_vec is None: continue

        # 5. Calculate Projections onto New Axes
        proj_primary = dot_product(relative_vec, primary_vec)
        proj_orthogonal = dot_product(relative_vec, orthogonal_vec)

        # Check if dot products are valid (vectors were valid)
        if proj_primary is None or proj_orthogonal is None:
             print(f"Warning: Could not calculate projection for object index {i}. Skipping.")
             continue


        # 6. Apply Condition: Primarily along primary axis
        if proj_primary > tolerance and proj_primary > abs(proj_orthogonal):
            found_objects.append(get_object_description(objects, i))

    return found_objects


# --- Specific Helper Functions for Each Direction ("View from Right") ---

def find_objects_left_from_new_view(data, ref_object_attrs):
    """Finds objects LEFT of ref_obj when viewing from RIGHT."""
    # New Left = Original Front | Orthogonal check: New Front = Original Left
    return find_objects_relative_direction_from_new_view(
        data, ref_object_attrs, 'front', 'left'
    )

def find_objects_right_from_new_view(data, ref_object_attrs):
    """Finds objects RIGHT of ref_obj when viewing from RIGHT."""
    # New Right = Original Behind | Orthogonal check: New Front = Original Left
    return find_objects_relative_direction_from_new_view(
        data, ref_object_attrs, 'behind', 'left'
    )

def find_objects_front_from_new_view(data, ref_object_attrs):
    """Finds objects FRONT of ref_obj when viewing from RIGHT."""
    # New Front = Original Left | Orthogonal check: New Left = Original Front
    return find_objects_relative_direction_from_new_view(
        data, ref_object_attrs, 'left', 'front'
    )

def find_objects_behind_from_new_view(data, ref_object_attrs):
    """Finds objects BEHIND ref_obj when viewing from RIGHT."""
    # New Behind = Original Right | Orthogonal check: New Left = Original Front
    return find_objects_relative_direction_from_new_view(
        data, ref_object_attrs, 'right', 'front'
    )


# --- Main Execution ---
if __name__ == "__main__":
    json_file_path ="/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000000.json"
    try:
        with open(json_file_path, 'r') as f:
            scene_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred loading JSON: {e}")
        exit()

    # Define the reference object
    ref_attrs = {"color": "purple", "shape": "cube"}
    ref_desc_str = get_object_description(scene_data['objects'], find_object_index_by_attrs(scene_data['objects'], ref_attrs))

    print(f"\n--- Running Spatial Queries (View from Right) Relative to: {ref_desc_str} ---")

    # --- Test LEFT ---
    print("\nQuery: What is to the LEFT?")
    left_objects = find_objects_left_from_new_view(scene_data, ref_attrs)
    if left_objects:
        print("Objects found to the LEFT:")
        for desc in left_objects: print(f"- {desc}")
    else:
        print("No objects found primarily to the LEFT.")

    # --- Test RIGHT ---
    print("\nQuery: What is to the RIGHT?")
    right_objects = find_objects_right_from_new_view(scene_data, ref_attrs)
    if right_objects:
        print("Objects found to the RIGHT:")
        for desc in right_objects: print(f"- {desc}")
    else:
        print("No objects found primarily to the RIGHT.")

    # --- Test FRONT ---
    print("\nQuery: What is in FRONT?")
    front_objects = find_objects_front_from_new_view(scene_data, ref_attrs)
    if front_objects:
        print("Objects found in FRONT:")
        for desc in front_objects: print(f"- {desc}")
    else:
        print("No objects found primarily in FRONT.")

    # --- Test BEHIND ---
    print("\nQuery: What is BEHIND?")
    behind_objects = find_objects_behind_from_new_view(scene_data, ref_attrs)
    if behind_objects:
        print("Objects found BEHIND:")
        for desc in behind_objects: print(f"- {desc}")
    else:
        print("No objects found primarily BEHIND.")


--- Running Spatial Queries (View from Right) Relative to: the large purple metal cube ---

Query: What is to the LEFT?
Objects found to the LEFT:
- the large blue rubber cylinder

Query: What is to the RIGHT?
Objects found to the RIGHT:
- the small red metal cube

Query: What is in FRONT?
Objects found in FRONT:
- the large gray metal cylinder
- the small gray metal cube
- the small cyan metal cube

Query: What is BEHIND?
Objects found BEHIND:
- the small green metal cylinder


In [3]:

# --- Main Execution ---
if __name__ == "__main__":
    json_file_path ="/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000003.json"
    try:
        with open(json_file_path, 'r') as f:
            scene_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred loading JSON: {e}")
        exit()

    # Define the reference object
    ref_attrs = {"color": "yellow", "shape": "sphere"}
    ref_desc_str = get_object_description(scene_data['objects'], find_object_index_by_attrs(scene_data['objects'], ref_attrs))

    print(f"\n--- Running Spatial Queries (View from Right) Relative to: {ref_desc_str} ---")

    # --- Test LEFT ---
    print("\nQuery: What is to the LEFT?")
    left_objects = find_objects_left_from_new_view(scene_data, ref_attrs)
    if left_objects:
        print("Objects found to the LEFT:")
        for desc in left_objects: print(f"- {desc}")
    else:
        print("No objects found primarily to the LEFT.")

    # --- Test RIGHT ---
    print("\nQuery: What is to the RIGHT?")
    right_objects = find_objects_right_from_new_view(scene_data, ref_attrs)
    if right_objects:
        print("Objects found to the RIGHT:")
        for desc in right_objects: print(f"- {desc}")
    else:
        print("No objects found primarily to the RIGHT.")

    # --- Test FRONT ---
    print("\nQuery: What is in FRONT?")
    front_objects = find_objects_front_from_new_view(scene_data, ref_attrs)
    if front_objects:
        print("Objects found in FRONT:")
        for desc in front_objects: print(f"- {desc}")
    else:
        print("No objects found primarily in FRONT.")

    # --- Test BEHIND ---
    print("\nQuery: What is BEHIND?")
    behind_objects = find_objects_behind_from_new_view(scene_data, ref_attrs)
    if behind_objects:
        print("Objects found BEHIND:")
        for desc in behind_objects: print(f"- {desc}")
    else:
        print("No objects found primarily BEHIND.")


--- Running Spatial Queries (View from Right) Relative to: the small yellow rubber sphere ---

Query: What is to the LEFT?
Objects found to the LEFT:
- the large brown rubber cylinder

Query: What is to the RIGHT?
Objects found to the RIGHT:
- the small yellow metal cylinder

Query: What is in FRONT?
Objects found in FRONT:
- the small red metal cylinder
- the small yellow metal cylinder

Query: What is BEHIND?
Objects found BEHIND:
- the small cyan rubber sphere
- the small purple metal cube


- Distribution of Answers are not skewed
- Negative Samples: Model should output 0
- 

In [3]:
# Script using 3D coordinates for viewpoint change (extended)
import json
import math

# --- Vector Math Helpers ---
def subtract_vectors(v1, v2):
    """Subtracts v2 from v1."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3: # Ensure 3D
        return None
    return [a - b for a, b in zip(v1, v2)]

def dot_product(v1, v2):
    """Calculates the dot product of v1 and v2."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3:
        return None
    return sum(a * b for a, b in zip(v1, v2))

def vector_magnitude(v):
    """Calculates the magnitude (length) of a vector."""
    if v is None or len(v)!=3: return 0
    return math.sqrt(sum(a * a for a in v))

def normalize_vector(v):
    """Normalizes a vector to unit length."""
    if v is None or len(v)!=3: return None
    mag = vector_magnitude(v)
    if mag == 0:
        return [0.0] * 3
    return [a / mag for a in v]

# --- Object Identification Helpers ---
def find_object_index_by_attrs(objects, attrs):
    """Finds the index of the first object matching all given attributes."""
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            return i
    return -1

def get_object_description(objects, index):
    """Generates a string description for an object at a given index."""
    if 0 <= index < len(objects) and isinstance(objects[index], dict):
        obj = objects[index]
        parts = [
            obj.get('size'), obj.get('color'), obj.get('material'), obj.get('shape')
        ]
        desc = ' '.join(filter(None, parts))
        return f"the {desc}" if desc else "unknown object"
    return "unknown object"

# --- Core Logic for Relative Direction Calculation ---
def find_objects_relative_direction_from_new_view(
    data,
    ref_object_attrs,
    primary_direction_original_key, # e.g., 'front' if query is new 'left' and new view makes original 'front' the new 'left'
    orthogonal_direction_original_key # e.g., 'left' if primary is new 'left'/'right' and new view makes original 'left' the new 'front'
    ):
    """
    General function to find objects in a specific relative direction
    from a reference object, based on transformed viewpoint axes.

    Args:
        data (dict): The loaded scene data.
        ref_object_attrs (dict): Attributes to find the reference object.
        primary_direction_original_key (str): The key in `data['directions']`
            that corresponds to the primary query direction in the *new* view.
        orthogonal_direction_original_key (str): The key in `data['directions']`
            that corresponds to an orthogonal direction (e.g., "front" of new view)
            in the *new* view, used for disambiguation.

    Returns:
        list: A list of descriptions of objects found in the specified direction.
    """
    if 'objects' not in data or 'directions' not in data:
        print("Error: 'objects' or 'directions' key not found in data.")
        return []

    objects = data['objects']
    directions = data['directions']

    ref_index = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_index == -1:
        print(f"Error: Reference object with attributes {ref_object_attrs} not found.")
        return []
    ref_obj = objects[ref_index]
    ref_coords = ref_obj.get('3d_coords')
    if ref_coords is None or len(ref_coords) != 3:
        print(f"Error: Missing or invalid 3D coordinates for reference object {ref_object_attrs}.")
        return []

    primary_vec = directions.get(primary_direction_original_key)
    orthogonal_vec = directions.get(orthogonal_direction_original_key)

    if primary_vec is None or orthogonal_vec is None or len(primary_vec)!=3 or len(orthogonal_vec)!=3:
        print(f"Error: Invalid direction vectors for keys '{primary_direction_original_key}' or '{orthogonal_direction_original_key}'.")
        return []

    found_objects = []
    tolerance = 0.1 # Minimum projection value onto primary axis

    for i, cand_obj in enumerate(objects):
        if i == ref_index: continue
        if not isinstance(cand_obj, dict): continue

        cand_coords = cand_obj.get('3d_coords')
        if cand_coords is None or len(cand_coords) != 3: continue

        relative_vec = subtract_vectors(cand_coords, ref_coords)
        if relative_vec is None: continue

        proj_primary = dot_product(relative_vec, primary_vec)
        proj_orthogonal = dot_product(relative_vec, orthogonal_vec)

        if proj_primary is None or proj_orthogonal is None:
             print(f"Warning: Could not calculate projection for object index {i}. Skipping.")
             continue

        # Condition: Primarily along primary_vec and more so than along orthogonal_vec
        if proj_primary > tolerance and proj_primary > abs(proj_orthogonal):
            found_objects.append(get_object_description(objects, i))

    return found_objects

# --- Viewpoint Configuration ---
# Defines how query directions (left, right, front, behind) in a NEW VIEWPOINT
# map to directions in the ORIGINAL scene's coordinate system.
# 'primary': The original direction key that aligns with the query direction in the new view.
# 'orthogonal': The original direction key that aligns with a perpendicular axis
#              (typically "front" or "left") in the new view, used for disambiguation.
VIEWPOINT_CONFIG = {
    'original': { # View from Original Front (standard view)
        'left':   {'primary': 'left',   'orthogonal': 'front'},
        'right':  {'primary': 'right',  'orthogonal': 'front'},
        'front':  {'primary': 'front',  'orthogonal': 'left'},
        'behind': {'primary': 'behind', 'orthogonal': 'left'},
    },
    'view_from_right': { # Camera is now on the original 'right' side, looking towards original 'left'
        # New Front = Original Left
        # New Left = Original Front
        'left':   {'primary': 'front',  'orthogonal': 'left'},
        'right':  {'primary': 'behind', 'orthogonal': 'left'},
        'front':  {'primary': 'left',   'orthogonal': 'front'},
        'behind': {'primary': 'right',  'orthogonal': 'front'},
    },
    'view_from_left': { # Camera is now on the original 'left' side, looking towards original 'right'
        # New Front = Original Right
        # New Left = Original Behind
        'left':   {'primary': 'behind', 'orthogonal': 'right'},
        'right':  {'primary': 'front',  'orthogonal': 'right'},
        'front':  {'primary': 'right',  'orthogonal': 'behind'},
        'behind': {'primary': 'left',   'orthogonal': 'behind'},
    },
    'view_from_back': { # Camera is now on the original 'behind' side, looking towards original 'front'
        # New Front = Original Behind
        # New Left = Original Right
        'left':   {'primary': 'right',  'orthogonal': 'behind'},
        'right':  {'primary': 'left',   'orthogonal': 'behind'},
        'front':  {'primary': 'behind', 'orthogonal': 'right'},
        'behind': {'primary': 'front',  'orthogonal': 'right'},
    }
}

def query_objects_from_viewpoint(data, ref_object_attrs, viewpoint_key, query_direction_key):
    """
    Finds objects in a specified direction from a reference object,
    considering a new viewpoint.

    Args:
        data (dict): The loaded scene data.
        ref_object_attrs (dict): Attributes to find the reference object.
        viewpoint_key (str): Key for the desired viewpoint (e.g., 'original', 'view_from_left').
        query_direction_key (str): Key for the query direction in the new viewpoint (e.g., 'left', 'front').

    Returns:
        list: A list of descriptions of objects found.
    """
    if viewpoint_key not in VIEWPOINT_CONFIG:
        print(f"Error: Viewpoint key '{viewpoint_key}' not recognized.")
        return []
    if query_direction_key not in VIEWPOINT_CONFIG[viewpoint_key]:
        print(f"Error: Query direction key '{query_direction_key}' not recognized for viewpoint '{viewpoint_key}'.")
        return []

    config = VIEWPOINT_CONFIG[viewpoint_key][query_direction_key]
    primary_original_key = config['primary']
    orthogonal_original_key = config['orthogonal']

    return find_objects_relative_direction_from_new_view(
        data,
        ref_object_attrs,
        primary_original_key,
        orthogonal_original_key
    )

# --- Main Execution ---
if __name__ == "__main__":
    # IMPORTANT: Replace this path with the actual path to your JSON file
    file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/"
    json_file_path =file_path+"CLEVR_new_000000.json" # Make sure this file exists or change path

    try:
        with open(json_file_path, 'r') as f:
            scene_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred loading JSON: {e}")
        exit()

    # Define the reference object (ensure this object exists in your JSON file)
    # For CLEVR_new_000000.json, a purple cube is usually object 0.
    ref_attrs = {"color": "purple", "shape": "cube"}
    ref_obj_idx = find_object_index_by_attrs(scene_data.get('objects', []), ref_attrs)

    if ref_obj_idx == -1:
        print(f"Error: Reference object with attributes {ref_attrs} not found in {json_file_path}.")
        ref_desc_str = "the specified reference object (not found)"
    else:
        ref_desc_str = get_object_description(scene_data['objects'], ref_obj_idx)


    viewpoints_to_test = ['original', 'view_from_right', 'view_from_left', 'view_from_back']
    query_directions_to_test = ['left', 'right', 'front', 'behind']

    for viewpoint in viewpoints_to_test:
        print(f"\n--- Queries from Viewpoint: {viewpoint.replace('_', ' ').title()} (Relative to: {ref_desc_str}) ---")
        if ref_obj_idx == -1 and viewpoint != 'original': # Skip if ref obj not found, except for maybe original view
             print(f"Skipping viewpoint {viewpoint} as reference object was not found.")
             continue

        for query_dir in query_directions_to_test:
            print(f"\n  Query: What is to the {query_dir.upper()} (from {viewpoint.replace('_', ' ')} view)?")
            found_objects = query_objects_from_viewpoint(scene_data, ref_attrs, viewpoint, query_dir)

            if found_objects:
                print(f"  Objects found to the {query_dir.upper()}:")
                for desc in found_objects:
                    print(f"  - {desc}")
            else:
                print(f"  No objects found primarily to the {query_dir.upper()}.")
        print("-" * 70)


--- Queries from Viewpoint: Original (Relative to: the large purple metal cube) ---

  Query: What is to the LEFT (from original view)?
  Objects found to the LEFT:
  - the large gray metal cylinder
  - the small gray metal cube
  - the small cyan metal cube

  Query: What is to the RIGHT (from original view)?
  Objects found to the RIGHT:
  - the small green metal cylinder

  Query: What is to the FRONT (from original view)?
  Objects found to the FRONT:
  - the large blue rubber cylinder

  Query: What is to the BEHIND (from original view)?
  Objects found to the BEHIND:
  - the small red metal cube
----------------------------------------------------------------------

--- Queries from Viewpoint: View From Right (Relative to: the large purple metal cube) ---

  Query: What is to the LEFT (from view from right view)?
  Objects found to the LEFT:
  - the large blue rubber cylinder

  Query: What is to the RIGHT (from view from right view)?
  Objects found to the RIGHT:
  - the small 

Error: Reference object with attributes {'color': 'purple', 'shape': 'cube'} not found in /Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000001.json.

--- Queries from Viewpoint: Original (Relative to: the specified reference object (not found)) ---

  Query: What is to the LEFT (from original view)?
Error: Reference object with attributes {'color': 'purple', 'shape': 'cube'} not found.
  No objects found primarily to the LEFT.

  Query: What is to the RIGHT (from original view)?
Error: Reference object with attributes {'color': 'purple', 'shape': 'cube'} not found.
  No objects found primarily to the RIGHT.

  Query: What is to the FRONT (from original view)?
Error: Reference object with attributes {'color': 'purple', 'shape': 'cube'} not found.
  No objects found primarily to the FRONT.

  Query: What is to the BEHIND (from original view)?
Error: Reference object with attributes {'color': 'purple', 'shape': 'cube'} not found.
  No objects found prima

In [8]:
import json
import math
import copy # For deepcopy in Q3

# --- Vector Math Helpers ---
def subtract_vectors(v1, v2):
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3: return None
    return [a - b for a, b in zip(v1, v2)]

def dot_product(v1, v2):
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3: return None
    return sum(a * b for a, b in zip(v1, v2))

def vector_magnitude(v):
    if v is None or len(v) != 3: return 0
    return math.sqrt(sum(a * a for a in v))

# --- Object Identification Helpers ---
def find_object_index_by_attrs(objects, attrs):
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            return i
    return -1

def find_objects_indices_by_attrs(objects, attrs):
    indices = []
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            indices.append(i)
    return indices

def get_object_description(objects, index):
    if not objects or not (0 <= index < len(objects)) or not isinstance(objects[index], dict):
        return "unknown object"
    obj = objects[index]
    parts = [obj.get('size'), obj.get('color'), obj.get('material'), obj.get('shape')]
    desc = ' '.join(filter(None, parts))
    return f"the {desc}" if desc else "an unknown object"

# --- Viewpoint and Direction Configuration ---
VIEWPOINT_CONFIG = {
    'original': { # View from Original Front (standard view)
        'left':   {'primary_key': 'left',   'orthogonal_key': 'front'},
        'right':  {'primary_key': 'right',  'orthogonal_key': 'front'},
        'front':  {'primary_key': 'front',  'orthogonal_key': 'left'},
        'behind': {'primary_key': 'behind', 'orthogonal_key': 'left'},
    },
    'view_from_right': {
        'left':   {'primary_key': 'front',  'orthogonal_key': 'left'},
        'right':  {'primary_key': 'behind', 'orthogonal_key': 'left'},
        'front':  {'primary_key': 'left',   'orthogonal_key': 'front'},
        'behind': {'primary_key': 'right',  'orthogonal_key': 'front'},
    },
    'view_from_left': {
        'left':   {'primary_key': 'behind', 'orthogonal_key': 'right'},
        'right':  {'primary_key': 'front',  'orthogonal_key': 'right'},
        'front':  {'primary_key': 'right',  'orthogonal_key': 'behind'},
        'behind': {'primary_key': 'left',   'orthogonal_key': 'behind'},
    },
    'view_from_back': {
        'left':   {'primary_key': 'right',  'orthogonal_key': 'behind'},
        'right':  {'primary_key': 'left',   'orthogonal_key': 'behind'},
        'front':  {'primary_key': 'behind', 'orthogonal_key': 'right'},
        'behind': {'primary_key': 'front',  'orthogonal_key': 'right'},
    }
}

# Tolerances for spatial comparisons
# Distance an object must be along the primary axis to be considered "in that direction"
HORIZONTAL_SEPARATION_TOLERANCE = 0.1
VERTICAL_SEPARATION_TOLERANCE = 0.1
# How far off-axis (horizontally) an object can be to be considered "directly" above/below
VERTICAL_ALIGNMENT_TOLERANCE = 0.5 # Max horizontal distance for an object to be "directly" above/below


#nearest object
def evaluate_q1_nearest_object(scene_data, viewpoint_str, ref_object_attrs):
    """
    Finds the object nearest to the reference object.
    Args:
        scene_data (dict): The scene data.
        viewpoint_str (str): Describes the viewpoint context (e.g., "front", "left_side").
                             Currently informational, doesn't change nearest calculation.
        ref_object_attrs (dict): Attributes to identify the reference object.
    Returns:
        str: Description of the nearest object, or an error message.
    """
    objects = scene_data.get('objects', [])
    if not objects:
        return "Error: No objects found in scene data."

    ref_idx = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_idx == -1:
        return f"Error: Reference object with attributes {ref_object_attrs} not found."

    ref_coords = objects[ref_idx].get('3d_coords')
    if not ref_coords:
        return f"Error: Reference object {get_object_description(objects, ref_idx)} has no 3D coordinates."

    min_dist = float('inf')
    nearest_obj_idx = -1

    for i, obj in enumerate(objects):
        if i == ref_idx:
            continue
        
        obj_coords = obj.get('3d_coords')
        if not obj_coords:
            continue

        relative_vec = subtract_vectors(obj_coords, ref_coords)
        if not relative_vec:
            continue
        
        dist = vector_magnitude(relative_vec)
        
        if dist < min_dist:
            min_dist = dist
            nearest_obj_idx = i
            
    if nearest_obj_idx != -1:
        # Viewpoint string is mostly for context matching the question.
        # print(f"(Context: Viewpoint set to {viewpoint_str})")
        return f"The nearest object to {get_object_description(objects, ref_idx)} is {get_object_description(objects, nearest_obj_idx)} (distance: {min_dist:.2f})."
    else:
        return f"No other objects found to determine the nearest to {get_object_description(objects, ref_idx)}."
    

#relative position (left/right/front/behind/above/below)
def _get_effective_view_params(scene_directions, viewpoint_key_for_config, query_direction_str):
    """
    Helper to get primary and orthogonal vectors based on viewpoint and query direction.
    Returns a dictionary with 'primary_vec', 'orthogonal_vec' (for horizontal),
    'is_vertical_query', and 'vertical_query_type' (for vertical).
    """
    params = {'is_vertical_query': False, 'primary_vec': None, 'orthogonal_vec': None, 'vertical_query_type': None}

    if query_direction_str in ['above', 'below']:
        params['is_vertical_query'] = True
        params['vertical_query_type'] = query_direction_str
        params['primary_vec'] = scene_directions.get(query_direction_str)
        if not params['primary_vec']:
            print(f"Warning: Direction vector for '{query_direction_str}' not found.")
            return None
    elif viewpoint_key_for_config in VIEWPOINT_CONFIG and \
         query_direction_str in VIEWPOINT_CONFIG[viewpoint_key_for_config]:
        config = VIEWPOINT_CONFIG[viewpoint_key_for_config][query_direction_str]
        primary_key = config['primary_key']
        orthogonal_key = config['orthogonal_key']
        params['primary_vec'] = scene_directions.get(primary_key)
        params['orthogonal_vec'] = scene_directions.get(orthogonal_key)
        if not params['primary_vec'] or not params['orthogonal_vec']:
            print(f"Warning: Direction vectors for primary/orthogonal keys ('{primary_key}', '{orthogonal_key}') not found.")
            return None
    else:
        print(f"Warning: Configuration not found for viewpoint '{viewpoint_key_for_config}' and query '{query_direction_str}'.")
        return None
    return params

def evaluate_q2_relative_position(scene_data, viewpoint_str, query_direction_str, ref_object_attrs):
    """
    Finds objects in a specified spatial relation from a reference object, considering a viewpoint.
    Args:
        scene_data (dict): The scene data.
        viewpoint_str (str): 'front', 'back', 'left_side', 'right_side'.
        query_direction_str (str): 'left', 'right', 'above', 'below', 'front', 'behind'.
        ref_object_attrs (dict): Attributes to identify the reference object.
    Returns:
        list: Descriptions of found objects, or an error message as a string.
    """
    objects = scene_data.get('objects', [])
    scene_directions = scene_data.get('directions', {})
    if not objects: return "Error: No objects in scene."
    if not scene_directions: return "Error: No directions in scene."

    ref_idx = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_idx == -1:
        return f"Error: Reference object {ref_object_attrs} not found."
    ref_coords = objects[ref_idx].get('3d_coords')
    if not ref_coords:
        return f"Error: Ref object {get_object_description(objects, ref_idx)} has no 3D coords."

    viewpoint_map = {
        'front': 'original', 'back': 'view_from_back',
        'left_side': 'view_from_left', 'right_side': 'view_from_right'
    }
    viewpoint_key_for_config = viewpoint_map.get(viewpoint_str)
    if not viewpoint_key_for_config:
        return f"Error: Invalid viewpoint_str '{viewpoint_str}'."

    view_params = _get_effective_view_params(scene_directions, viewpoint_key_for_config, query_direction_str)
    if not view_params or not view_params['primary_vec']:
        return f"Error: Could not determine view parameters for query."
        
    found_objects_descs = []

    for i, cand_obj in enumerate(objects):
        if i == ref_idx: continue
        cand_coords = cand_obj.get('3d_coords')
        if not cand_coords: continue

        relative_vec = subtract_vectors(cand_coords, ref_coords)
        if not relative_vec: continue

        if view_params['is_vertical_query']:
            # Vertical query (above/below)
            proj_primary = dot_product(relative_vec, view_params['primary_vec'])
            
            # For 'above', relative_vec[2] should be positive. directions['above'] is (0,0,1). proj_primary = relative_vec[2].
            # For 'below', relative_vec[2] should be negative. directions['below'] is (0,0,-1). proj_primary = -relative_vec[2].
            # So, in both cases, we want proj_primary to be positive relative to the direction vector.
            if proj_primary > VERTICAL_SEPARATION_TOLERANCE:
                horizontal_distance_vec = [relative_vec[0], relative_vec[1], 0.0]
                if vector_magnitude(horizontal_distance_vec) < VERTICAL_ALIGNMENT_TOLERANCE:
                    found_objects_descs.append(get_object_description(objects, i))
        else:
            # Horizontal query (left/right/front/behind)
            if not view_params['orthogonal_vec']: # Should be set for horizontal
                return "Error: Orthogonal vector missing for horizontal query."
            proj_primary = dot_product(relative_vec, view_params['primary_vec'])
            proj_orthogonal = dot_product(relative_vec, view_params['orthogonal_vec'])
            
            if proj_primary > HORIZONTAL_SEPARATION_TOLERANCE and \
               proj_primary > abs(proj_orthogonal):
                found_objects_descs.append(get_object_description(objects, i))
                
    ref_desc = get_object_description(objects, ref_idx)
    if found_objects_descs:
        return found_objects_descs
    else:
        return f"No objects found {query_direction_str} of {ref_desc} from the {viewpoint_str} viewpoint."


#swap positions and count relative objects
def evaluate_q3_swap_and_count_relative(scene_data_original, viewpoint_str, 
                                        swap_group1_attrs, swap_group2_attrs, 
                                        query_direction_str, ref_object_attrs_after_swap):
    """
    Swaps positions of objects from two groups, then counts objects in a relative
    position to a reference object in the modified scene.
    Args:
        scene_data_original (dict): The original scene data.
        viewpoint_str (str): Viewpoint for the final query ('front', 'back', etc.).
        swap_group1_attrs (dict): Attributes for the first group of objects to swap (e.g., {'color':'red'}).
        swap_group2_attrs (dict): Attributes for the second group of objects to swap (e.g., {'shape':'cube'}).
        query_direction_str (str): Spatial relation for counting ('left', 'behind', 'front').
        ref_object_attrs_after_swap (dict): Attributes to find ref object *after* potential swap.
    Returns:
        str: A message with the count of objects, or an error message.
    """
    if not scene_data_original.get('objects'):
        return "Error: Original scene data has no objects."

    # 1. Deep copy the scene data to modify it
    scene_data_modified = copy.deepcopy(scene_data_original)
    objects_modified = scene_data_modified['objects']

    # 2. Perform the swap
    group1_indices = find_objects_indices_by_attrs(objects_modified, swap_group1_attrs)
    group2_indices = find_objects_indices_by_attrs(objects_modified, swap_group2_attrs)
    
    # Avoid swapping an object with itself if it's in both groups by ensuring distinct indices
    # This simple filtering works if an object can't be swapped with another from its own original selection list.
    # For more complex overlap, a more sophisticated distinct element selection would be needed.
    # Here, we ensure we are trying to swap elements from two *potentially* different sets.
    # A quick way to handle overlap: remove common indices from one group for the swap pairing.
    
    # Filter out indices that might be common to prevent an object being swapped with itself if lists overlap significantly
    # and to ensure we are swapping between distinct conceptual groups as much as possible.
    # The current find_objects_indices_by_attrs doesn't inherently prevent an object from being in both lists
    # if e.g. swap_group1_attrs = {'color': 'red'} and swap_group2_attrs = {'material': 'metal'}
    # and a red metal object exists.
    
    # For swapping positions, we just need the 3D coords.
    # Let's consider a direct pairwise swap of coordinates.
    
    num_to_swap = min(len(group1_indices), len(group2_indices))
    swapped_info = []

    if num_to_swap > 0:
        print(f"Attempting to swap {num_to_swap} pairs of objects...")
        # Store original coordinates before swapping to avoid issues if an object is in both lists
        # and gets its coordinates changed before being used as a source for another swap.
        group1_coords_original = [objects_modified[i]['3d_coords'] for i in group1_indices[:num_to_swap]]
        group2_coords_original = [objects_modified[i]['3d_coords'] for i in group2_indices[:num_to_swap]]

        for i in range(num_to_swap):
            idx1 = group1_indices[i]
            idx2 = group2_indices[i]

            # Check if we are trying to swap an object with itself through list overlap
            # This check is simple; complex overlaps need more robust handling
            if idx1 == idx2:
                print(f"Skipping swap for object index {idx1} as it's the same in both identified groups for this pair.")
                continue

            # Swap 3d_coords using the stored original values
            objects_modified[idx1]['3d_coords'] = group2_coords_original[i]
            objects_modified[idx2]['3d_coords'] = group1_coords_original[i]
            
            # For completeness, if pixel_coords were important for other reasons, they'd be swapped too.
            # if 'pixel_coords' in objects_modified[idx1] and 'pixel_coords' in objects_modified[idx2]:
            #     temp_px = objects_modified[idx1]['pixel_coords']
            #     objects_modified[idx1]['pixel_coords'] = objects_modified[idx2]['pixel_coords']
            #     objects_modified[idx2]['pixel_coords'] = temp_px
            
            swapped_info.append(f"Swapped position of {get_object_description(scene_data_original['objects'], idx1)} "
                                f"with {get_object_description(scene_data_original['objects'], idx2)}")
        if swapped_info:
            print("Swap Report:")
            for info in swapped_info: print(f"- {info}")
    else:
        print("No objects eligible for swapping based on criteria, or groups were empty.")

    # 3. Perform the relative position query on the modified scene
    # This part uses logic similar to evaluate_q2_relative_position
    
    # Find reference object in the *modified* scene
    ref_idx_after_swap = find_object_index_by_attrs(objects_modified, ref_object_attrs_after_swap)
    if ref_idx_after_swap == -1:
        return f"Error: Reference object {ref_object_attrs_after_swap} not found in the modified scene."
    
    ref_coords_after_swap = objects_modified[ref_idx_after_swap].get('3d_coords')
    if not ref_coords_after_swap:
        return f"Error: Ref object (post-swap) {get_object_description(objects_modified, ref_idx_after_swap)} has no 3D coords."

    viewpoint_map = {
        'front': 'original', 'back': 'view_from_back',
        'left_side': 'view_from_left', 'right_side': 'view_from_right'
    }
    viewpoint_key_for_config = viewpoint_map.get(viewpoint_str)
    if not viewpoint_key_for_config: return f"Error: Invalid viewpoint_str '{viewpoint_str}'."

    scene_directions_modified = scene_data_modified.get('directions', {})
    if not scene_directions_modified: return "Error: No directions in modified scene."

    view_params = _get_effective_view_params(scene_directions_modified, viewpoint_key_for_config, query_direction_str)
    if not view_params or not view_params['primary_vec']:
        return f"Error: Could not determine view parameters for query in modified scene."

    count = 0
    for i, cand_obj in enumerate(objects_modified):
        if i == ref_idx_after_swap: continue
        cand_coords = cand_obj.get('3d_coords')
        if not cand_coords: continue

        relative_vec = subtract_vectors(cand_coords, ref_coords_after_swap)
        if not relative_vec: continue
        
        # Only horizontal check needed for Q3 as per example "left of/behind/in front of"
        if view_params['is_vertical_query']:
            print("Warning: Q3 example implies horizontal query, but got vertical. Counting anyway.")
            proj_primary = dot_product(relative_vec, view_params['primary_vec'])
            if proj_primary > VERTICAL_SEPARATION_TOLERANCE:
                horizontal_distance_vec = [relative_vec[0], relative_vec[1], 0.0]
                if vector_magnitude(horizontal_distance_vec) < VERTICAL_ALIGNMENT_TOLERANCE:
                    count += 1
        else: # Horizontal
            if not view_params['orthogonal_vec']:
                 return "Error: Orthogonal vector missing for horizontal query in modified scene."
            proj_primary = dot_product(relative_vec, view_params['primary_vec'])
            proj_orthogonal = dot_product(relative_vec, view_params['orthogonal_vec'])
            
            if proj_primary > HORIZONTAL_SEPARATION_TOLERANCE and \
               proj_primary > abs(proj_orthogonal):
                count += 1
                
    ref_desc_after_swap = get_object_description(objects_modified, ref_idx_after_swap)
    return (f"After swapping '{swap_group1_attrs}' with '{swap_group2_attrs}', "
            f"there are {count} objects {query_direction_str} of {ref_desc_after_swap} "
            f"from the {viewpoint_str} viewpoint.")
# (Keep all previous helper functions: subtract_vectors, dot_product, vector_magnitude,
# find_object_index_by_attrs, find_objects_indices_by_attrs, get_object_description,
# VIEWPOINT_CONFIG, HORIZONTAL_SEPARATION_TOLERANCE, etc.)



In [10]:
if __name__ == "__main__":
    # Load your JSON data (make sure the file path is correct)
    #file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/"
    #json_file_path = file_path + "CLEVR_new_0000001.json" # Path to your JSON
    json_file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000000.json"
    try:
        with open(json_file_path, 'r') as f:
            scene_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        exit()

    print("--- Question Type 1: Nearest Object ---")
    # Q1: Change the viewpoint to front. Which object is nearest to the large purple metal cube?
    ref_attrs_q1 = {"size": "large", "color": "purple", "material": "metal", "shape": "cube"}
    result_q1 = evaluate_q1_nearest_object(scene_data, "front", ref_attrs_q1)
    print(f"Q1 Result: {result_q1}")
    print("-" * 50)

    print("\n--- Question Type 2: Relative Position ---")
    # Q2: Change the viewpoint to right_side. Which object is to the left of the small red metal cube?
    ref_attrs_q2 = {"size": "small", "color": "red", "material": "metal", "shape": "cube"}
    result_q2 = evaluate_q2_relative_position(scene_data, "right_side", "left", ref_attrs_q2)
    print(f"Q2 Result (left from right_side view):")
    if isinstance(result_q2, list):
        for desc in result_q2: print(f"- {desc}")
    else:
        print(result_q2)

    # Q2 Example: Above
    # Assuming the 'large gray metal cylinder' is object 2 at [-0.87, -0.29, 0.7]
    # And the 'small red metal cube' is object 1 at [-2.93, 2.39, 0.35]
    # Cube is below cylinder.
    ref_attrs_q2_above = {"size": "small", "color": "red", "material": "metal", "shape": "cube"} # the red cube
    result_q2_above = evaluate_q2_relative_position(scene_data, "front", "above", ref_attrs_q2_above)
    print(f"\nQ2 Result (above of the small red metal cube from front view):")
    if isinstance(result_q2_above, list):
        if result_q2_above:
            for desc in result_q2_above: print(f"- {desc}")
        else:
            print("No objects found directly above.")
    else:
        print(result_q2_above)
    print("-" * 50)


    print("\n--- Question Type 3: Swap and Count Relative ---")
    # Q3: Swap positions of all red objects with all cube objects.
    # How many objects are to the left of the (originally) large purple metal cube from the front view now?
    
    swap_g1 = {'color': 'red'}      # Find all red objects
    swap_g2 = {'shape': 'cube'}     # Find all cube objects
    
    
    ref_attrs_q3 = {"size": "large", "color": "gray", "material": "metal", "shape": "cylinder"}
    
    result_q3 = evaluate_q3_swap_and_count_relative(
        scene_data, "front", 
        swap_g1, swap_g2,
        "left", ref_attrs_q3
    )
    print(f"Q3 Result: {result_q3}")
    
    # Another Q3 Example: Reference object *is* part of a swap group
    # If we swap red objects (obj 1) with cube objects (obj 0, 1, 5, 6).
    # After swap, what is to the left of the object *now described as* "small red metal cube"?
    # The object that ends up being "small red metal cube" might be the one that moved into its original position,
    # or the original one if its color wasn't changed by the swap.
    # Since we only swap positions, object attributes (color, shape) remain with the object instance.
    # So "the small red metal cube" is still object 1, just at a new location.
    ref_attrs_q3_dynamic = {"size": "small", "color": "red", "material": "metal", "shape": "cube"}
    result_q3_dynamic = evaluate_q3_swap_and_count_relative(
        scene_data, "front",
        {'color': 'green'}, # Swap green objects (obj 4)
        {'shape': 'cylinder'}, # with cylinder objects (obj 2, 3, 4)
        "behind", ref_attrs_q3_dynamic # query relative to red cube (obj 1), which is not part of this swap
    )
    print(f"\nQ3 Result (dynamic ref, different swap): {result_q3_dynamic}")
    print("-" * 50)

--- Question Type 1: Nearest Object ---
Q1 Result: The nearest object to the large purple metal cube is the small red metal cube (distance: 2.17).
--------------------------------------------------

--- Question Type 2: Relative Position ---
Q2 Result (left from right_side view):
- the large purple metal cube
- the large gray metal cylinder
- the large blue rubber cylinder
- the small green metal cylinder
- the small gray metal cube

Q2 Result (above of the small red metal cube from front view):
No objects found above of the small red metal cube from the front viewpoint.
--------------------------------------------------

--- Question Type 3: Swap and Count Relative ---
Attempting to swap 1 pairs of objects...
Swap Report:
- Swapped position of the small red metal cube with the large purple metal cube
Q3 Result: After swapping '{'color': 'red'}' with '{'shape': 'cube'}', there are 1 objects left of the large gray metal cylinder from the front viewpoint.
Attempting to swap 1 pairs of ob

In [11]:
if __name__ == "__main__":
    # Load your JSON data (make sure the file path is correct)
    # Ensure this path points to your CLEVR_new_000001.json file
    json_file_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000001.json"
    try:
        with open(json_file_path, 'r') as f:
            scene_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        exit()

    print("--- Question Type 1: Nearest Object (CLEVR_new_000001.json) ---")
    # Q1: Example for CLEVR_new_000001.json
    # Which object is nearest to the large yellow rubber sphere? (Object 0)
    ref_attrs_q1 = {"size": "large", "color": "yellow", "material": "rubber", "shape": "sphere"}
    result_q1 = evaluate_q1_nearest_object(scene_data, "front", ref_attrs_q1)
    print(f"Q1 Result: {result_q1}")
    print("-" * 70)

    print("\n--- Question Type 2: Relative Position (CLEVR_new_000001.json) ---")
    # Q2: Example for CLEVR_new_000001.json
    # Change the viewpoint to left_side. Which object is to the right of the small blue metal cube? (Object 4)
    ref_attrs_q2 = {"size": "small", "color": "blue", "material": "metal", "shape": "cube"}
    result_q2 = evaluate_q2_relative_position(scene_data, "left_side", "right", ref_attrs_q2)
    print(f"Q2 Result (right from left_side view of the small blue metal cube):")
    if isinstance(result_q2, list) and result_q2: # Check if list and not empty
        for desc in result_q2: print(f"- {desc}")
    elif isinstance(result_q2, list) and not result_q2: # Check if list and empty
        print(f"No objects found to the right from the left_side view of {get_object_description(scene_data.get('objects',[]), find_object_index_by_attrs(scene_data.get('objects',[]), ref_attrs_q2))}.")
    else: # It's an error string
        print(result_q2)

    # Q2 Example: Below
    # Which object is below the large red rubber cylinder? (Object 5)
    ref_attrs_q2_below = {"size": "large", "color": "red", "material": "rubber", "shape": "cylinder"}
    result_q2_below = evaluate_q2_relative_position(scene_data, "front", "below", ref_attrs_q2_below)
    print(f"\nQ2 Result (below the large red rubber cylinder from front view):")
    if isinstance(result_q2_below, list) and result_q2_below:
        for desc in result_q2_below: print(f"- {desc}")
    elif isinstance(result_q2_below, list) and not result_q2_below:
        print(f"No objects found directly below {get_object_description(scene_data.get('objects',[]), find_object_index_by_attrs(scene_data.get('objects',[]), ref_attrs_q2_below))}.")
    else:
        print(result_q2_below)
    print("-" * 70)

    print("\n--- Question Type 3: Swap and Count Relative (CLEVR_new_000001.json) ---")
    # Q3: Example for CLEVR_new_000001.json
    # Swap positions of all yellow objects with all sphere objects.
    # How many objects are to the front of the (originally) large red rubber cylinder from the right_side view now?

    swap_g1 = {'color': 'yellow'}  # Yellow objects (obj 0: large yellow rubber sphere, obj 1: small yellow rubber cube)
    swap_g2 = {'shape': 'sphere'}  # Sphere objects (obj 0: large yellow rubber sphere, obj 3: small brown metal sphere)
    
    # Reference object for Q3: the large red rubber cylinder (Object 5).
    # This object is not yellow and not a sphere, so its position should not change from these swaps.
    ref_attrs_q3 = {"size": "large", "color": "red", "material": "rubber", "shape": "cylinder"}
    
    result_q3 = evaluate_q3_swap_and_count_relative(
        scene_data, "right_side", 
        swap_g1, swap_g2,
        "front", ref_attrs_q3
    )
    print(f"Q3 Result: {result_q3}")
    
    # Another Q3 Example: Reference object IS part of a swap group.
    # Swap all 'brown' objects with all 'cylinder' objects.
    
    swap_g3_color = {'color': 'brown'} # obj 2 (small brown rubber cylinder), obj 3 (small brown metal sphere)
    swap_g3_shape = {'shape': 'cylinder'} # obj 2 (small brown rubber cylinder), obj 5 (large red rubber cylinder)

   
    ref_attrs_q3_dynamic = {"size": "small", "color": "brown", "material": "rubber", "shape": "cylinder"}
    
    result_q3_dynamic = evaluate_q3_swap_and_count_relative(
        scene_data, "front",
        swap_g3_color, 
        swap_g3_shape,
        "behind", ref_attrs_q3_dynamic 
    )
    print(f"\nQ3 Result (dynamic ref, brown/cylinder swap): {result_q3_dynamic}")
    print("-" * 70)

--- Question Type 1: Nearest Object (CLEVR_new_000001.json) ---
Q1 Result: The nearest object to the large yellow rubber sphere is the small yellow rubber cube (distance: 1.99).
----------------------------------------------------------------------

--- Question Type 2: Relative Position (CLEVR_new_000001.json) ---
Q2 Result (right from left_side view of the small blue metal cube):
No objects found right of the small blue metal cube from the left_side viewpoint.

Q2 Result (below the large red rubber cylinder from front view):
No objects found below of the large red rubber cylinder from the front viewpoint.
----------------------------------------------------------------------

--- Question Type 3: Swap and Count Relative (CLEVR_new_000001.json) ---
Attempting to swap 2 pairs of objects...
Skipping swap for object index 0 as it's the same in both identified groups for this pair.
Swap Report:
- Swapped position of the small yellow rubber cube with the small brown metal sphere
Q3 Result:

In [12]:
import json
import math
import copy # For deepcopy in Q3

# --- Vector Math Helpers ---
def subtract_vectors(v1, v2):
    """Subtracts v2 from v1."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3: # Ensure 3D
        return None
    return [a - b for a, b in zip(v1, v2)]

def dot_product(v1, v2):
    """Calculates the dot product of v1 and v2."""
    if v1 is None or v2 is None or len(v1) != 3 or len(v2) != 3:
        return None
    return sum(a * b for a, b in zip(v1, v2))

def vector_magnitude(v):
    """Calculates the magnitude (length) of a vector."""
    if v is None or len(v) != 3: return 0.0 # Return float for consistency
    return math.sqrt(sum(a * a for a in v))

# --- Object Identification Helpers ---
def find_object_index_by_attrs(objects, attrs):
    """Finds the index of the first object matching all given attributes."""
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            return i
    return -1

def find_objects_indices_by_attrs(objects, attrs):
    """Finds the indices of all objects matching all given attributes."""
    indices = []
    for i, obj in enumerate(objects):
        if not isinstance(obj, dict): continue
        match = True
        for key, value in attrs.items():
            if obj.get(key) != value:
                match = False
                break
        if match:
            indices.append(i)
    return indices

def get_object_description(objects, index):
    """Generates a string description for an object at a given index."""
    if not objects or not (0 <= index < len(objects)) or not isinstance(objects[index], dict):
        return "unknown object"
    obj = objects[index]
    parts = [obj.get('size'), obj.get('color'), obj.get('material'), obj.get('shape')]
    desc = ' '.join(filter(None, parts)) # filter(None, parts) removes None or empty strings
    return f"the {desc}" if desc else "an undescribed object"

# --- Viewpoint and Direction Configuration ---
VIEWPOINT_CONFIG = {
    'original': { # View from Original Front (standard view)
        'left':   {'primary_key': 'left',   'orthogonal_key': 'front'},
        'right':  {'primary_key': 'right',  'orthogonal_key': 'front'},
        'front':  {'primary_key': 'front',  'orthogonal_key': 'left'},
        'behind': {'primary_key': 'behind', 'orthogonal_key': 'left'},
    },
    'view_from_right': { # Camera is now on the original 'right' side, looking towards original 'left'
        'left':   {'primary_key': 'front',  'orthogonal_key': 'left'},
        'right':  {'primary_key': 'behind', 'orthogonal_key': 'left'},
        'front':  {'primary_key': 'left',   'orthogonal_key': 'front'},
        'behind': {'primary_key': 'right',  'orthogonal_key': 'front'},
    },
    'view_from_left': { # Camera is now on the original 'left' side, looking towards original 'right'
        'left':   {'primary_key': 'behind', 'orthogonal_key': 'right'},
        'right':  {'primary_key': 'front',  'orthogonal_key': 'right'},
        'front':  {'primary_key': 'right',  'orthogonal_key': 'behind'},
        'behind': {'primary_key': 'left',   'orthogonal_key': 'behind'},
    },
    'view_from_back': { # Camera is now on the original 'behind' side, looking towards original 'front'
        'left':   {'primary_key': 'right',  'orthogonal_key': 'behind'},
        'right':  {'primary_key': 'left',   'orthogonal_key': 'behind'},
        'front':  {'primary_key': 'behind', 'orthogonal_key': 'right'},
        'behind': {'primary_key': 'front',  'orthogonal_key': 'right'},
    }
}

# Tolerance for horizontal spatial comparisons
# Distance an object must be along the primary axis to be considered "in that direction"
HORIZONTAL_SEPARATION_TOLERANCE = 0.1

# --- Helper for View Parameters (Horizontal Only) ---
def _get_effective_view_params(scene_directions, viewpoint_key_for_config, query_direction_str):
    """
    Helper to get primary and orthogonal vectors based on viewpoint and query direction
    FOR HORIZONTAL QUERIES ONLY (left, right, front, behind).
    """
    params = {'primary_vec': None, 'orthogonal_vec': None}
    if viewpoint_key_for_config in VIEWPOINT_CONFIG and \
       query_direction_str in VIEWPOINT_CONFIG[viewpoint_key_for_config]:
        config = VIEWPOINT_CONFIG[viewpoint_key_for_config][query_direction_str]
        primary_key = config['primary_key']
        orthogonal_key = config['orthogonal_key']
        params['primary_vec'] = scene_directions.get(primary_key)
        params['orthogonal_vec'] = scene_directions.get(orthogonal_key)
        if not params['primary_vec'] or not params['orthogonal_vec']:
            # print(f"Debug: Direction vectors for p_key:'{primary_key}' or o_key:'{orthogonal_key}' not found in scene_directions.")
            return None # Indicate error
    else:
        # print(f"Debug: Horizontal config not found for viewpoint '{viewpoint_key_for_config}', query '{query_direction_str}'.")
        return None # Indicate error
    return params

# --- Evaluation Function 1: Nearest Object ---
def evaluate_q1_nearest_object(scene_data, viewpoint_str, ref_object_attrs):
    """
    Finds the object nearest to the reference object.
    Args:
        scene_data (dict): The scene data.
        viewpoint_str (str): Describes the viewpoint context (e.g., "front", "left_side").
                             Currently informational, doesn't change nearest calculation.
        ref_object_attrs (dict): Attributes to identify the reference object.
    Returns:
        str: Description of the nearest object, or an error message.
    """
    objects = scene_data.get('objects', [])
    if not objects:
        return "Error: No objects found in scene data."

    ref_idx = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_idx == -1:
        return f"Error: Reference object with attributes {ref_object_attrs} not found."

    ref_coords = objects[ref_idx].get('3d_coords')
    if not ref_coords:
        return f"Error: Reference object {get_object_description(objects, ref_idx)} has no 3D coordinates."

    min_dist = float('inf')
    nearest_obj_idx = -1
    num_other_objects = 0

    for i, obj in enumerate(objects):
        if i == ref_idx:
            continue
        
        obj_coords = obj.get('3d_coords')
        if not obj_coords:
            continue
        num_other_objects +=1

        relative_vec = subtract_vectors(obj_coords, ref_coords)
        if not relative_vec: # Should not happen if coords are valid
            continue
        
        dist = vector_magnitude(relative_vec)
        
        if dist < min_dist:
            min_dist = dist
            nearest_obj_idx = i
            
    if nearest_obj_idx != -1:
        return (f"The nearest object to {get_object_description(objects, ref_idx)} "
                f"is {get_object_description(objects, nearest_obj_idx)} (distance: {min_dist:.2f}). "
                f"(Context: Viewpoint '{viewpoint_str}')")
    elif num_other_objects == 0:
        return f"No other objects found in the scene to compare with {get_object_description(objects, ref_idx)}."
    else: # This case should ideally not be reached if other objects exist but somehow min_dist wasn't updated.
        return f"Could not determine the nearest object to {get_object_description(objects, ref_idx)} among other objects."

# --- Evaluation Function 2: Relative Position (Horizontal Only) ---
def evaluate_q2_relative_position(scene_data, viewpoint_str, query_direction_str, ref_object_attrs):
    """
    Finds objects in a specified HORIZONTAL spatial relation (left, right, front, behind)
    from a reference object, considering a viewpoint.
    Args:
        scene_data (dict): The scene data.
        viewpoint_str (str): 'front', 'back', 'left_side', 'right_side'.
        query_direction_str (str): 'left', 'right', 'front', 'behind'.
        ref_object_attrs (dict): Attributes to identify the reference object.
    Returns:
        list: Descriptions of found objects, or an error message as a string.
    """
    if query_direction_str in ['above', 'below']:
        return (f"Error: '{query_direction_str}' queries are not supported. "
                f"This function only handles 'left', 'right', 'front', 'behind'.")

    objects = scene_data.get('objects', [])
    scene_directions = scene_data.get('directions', {})
    if not objects: return "Error: No objects in scene."
    if not scene_directions: return "Error: No directions in scene."

    ref_idx = find_object_index_by_attrs(objects, ref_object_attrs)
    if ref_idx == -1:
        return f"Error: Reference object {ref_object_attrs} not found."
    ref_coords = objects[ref_idx].get('3d_coords')
    if not ref_coords:
        return f"Error: Ref object {get_object_description(objects, ref_idx)} has no 3D coords."

    viewpoint_map = {
        'front': 'original', 'back': 'view_from_back',
        'left_side': 'view_from_left', 'right_side': 'view_from_right'
    }
    viewpoint_key_for_config = viewpoint_map.get(viewpoint_str)
    if not viewpoint_key_for_config: # Should not happen if viewpoint_str is from a controlled set
        return f"Error: Invalid viewpoint_str '{viewpoint_str}' for mapping."

    view_params = _get_effective_view_params(scene_directions, viewpoint_key_for_config, query_direction_str)
    
    if not view_params or not view_params.get('primary_vec') or not view_params.get('orthogonal_vec'):
        return (f"Error: Could not determine valid view parameters for query '{query_direction_str}' "
                f"from viewpoint '{viewpoint_str}'. Check viewpoint and direction names.")
        
    found_objects_descs = []

    for i, cand_obj in enumerate(objects):
        if i == ref_idx: continue
        cand_coords = cand_obj.get('3d_coords')
        if not cand_coords: continue

        relative_vec = subtract_vectors(cand_coords, ref_coords)
        if not relative_vec: continue

        proj_primary = dot_product(relative_vec, view_params['primary_vec'])
        proj_orthogonal = dot_product(relative_vec, view_params['orthogonal_vec'])
        
        if proj_primary > HORIZONTAL_SEPARATION_TOLERANCE and \
           proj_primary > abs(proj_orthogonal):
            found_objects_descs.append(get_object_description(objects, i))
                
    if found_objects_descs:
        return found_objects_descs
    else:
        # Return a specific string instead of an empty list to distinguish from errors
        ref_obj_desc = get_object_description(objects, ref_idx)
        return f"No objects found {query_direction_str} of {ref_obj_desc} from the {viewpoint_str} viewpoint."


# --- Evaluation Function 3: Swap and Count Relative ---
def evaluate_q3_swap_and_count_relative(scene_data_original, viewpoint_str,
                                        swap_group1_attrs, swap_group2_attrs,
                                        query_direction_str, ref_object_attrs_after_swap):
    """
    Swaps positions of objects from two groups, then counts objects in a horizontal relative
    position to a reference object in the modified scene.
    Args:
        scene_data_original (dict): The original scene data.
        viewpoint_str (str): Viewpoint for the final query ('front', 'back', etc.).
        swap_group1_attrs (dict): Attributes for the first group of objects to swap.
        swap_group2_attrs (dict): Attributes for the second group of objects to swap.
        query_direction_str (str): Horizontal spatial relation ('left', 'behind', 'front', 'right').
        ref_object_attrs_after_swap (dict): Attributes to find ref object *after* potential swap.
    Returns:
        str: A message with the count of objects, or an error message.
    """
    if query_direction_str in ['above', 'below']:
        return (f"Error: Q3 currently supports horizontal queries only (left, right, front, behind). "
                f"Got '{query_direction_str}'.")

    if not scene_data_original.get('objects'):
        return "Error: Original scene data has no objects."

    scene_data_modified = copy.deepcopy(scene_data_original)
    objects_modified = scene_data_modified['objects']

    group1_indices = find_objects_indices_by_attrs(objects_modified, swap_group1_attrs)
    group2_indices = find_objects_indices_by_attrs(objects_modified, swap_group2_attrs)
    
    num_to_swap = min(len(group1_indices), len(group2_indices))
    # swapped_info_details = [] # Uncomment for detailed swap logging

    if num_to_swap > 0:
        group1_coords_to_swap = [copy.deepcopy(objects_modified[idx]['3d_coords']) for idx in group1_indices[:num_to_swap]]
        group2_coords_to_swap = [copy.deepcopy(objects_modified[idx]['3d_coords']) for idx in group2_indices[:num_to_swap]]

        for i in range(num_to_swap):
            idx1 = group1_indices[i]
            idx2 = group2_indices[i]
            
            if idx1 == idx2: 
                # swapped_info_details.append(f"Skipped swapping object index {idx1} with itself.")
                continue
            
            # original_desc_obj1 = get_object_description(scene_data_original['objects'], idx1)
            # original_desc_obj2 = get_object_description(scene_data_original['objects'], idx2)

            objects_modified[idx1]['3d_coords'] = group2_coords_to_swap[i]
            objects_modified[idx2]['3d_coords'] = group1_coords_to_swap[i]
            
            # swapped_info_details.append(f"Position of ({original_desc_obj1}) swapped with ({original_desc_obj2})")
        
        # if swapped_info_details: # Uncomment for verbose logging
        #     print("Swap Report:")
        #     for info_detail in swapped_info_details: print(f"- {info_detail}")


    ref_idx_after_swap = find_object_index_by_attrs(objects_modified, ref_object_attrs_after_swap)
    if ref_idx_after_swap == -1:
        return f"Error: Reference object {ref_object_attrs_after_swap} not found in the modified scene."
    
    ref_coords_after_swap = objects_modified[ref_idx_after_swap].get('3d_coords')
    if not ref_coords_after_swap:
        return f"Error: Ref object (post-swap) {get_object_description(objects_modified, ref_idx_after_swap)} has no 3D coords."

    viewpoint_map = {
        'front': 'original', 'back': 'view_from_back',
        'left_side': 'view_from_left', 'right_side': 'view_from_right'
    }
    viewpoint_key_for_config = viewpoint_map.get(viewpoint_str)
    if not viewpoint_key_for_config: return f"Error: Invalid viewpoint_str '{viewpoint_str}' for mapping."

    scene_directions_modified = scene_data_modified.get('directions', {})
    if not scene_directions_modified: return "Error: No directions in modified scene."

    view_params = _get_effective_view_params(scene_directions_modified, viewpoint_key_for_config, query_direction_str)
    
    if not view_params or not view_params.get('primary_vec') or not view_params.get('orthogonal_vec'):
        return (f"Error: Could not determine valid view parameters for query '{query_direction_str}' "
                f"from viewpoint '{viewpoint_str}' in modified scene.")

    count = 0
    for i, cand_obj in enumerate(objects_modified):
        if i == ref_idx_after_swap: continue
        cand_coords = cand_obj.get('3d_coords')
        if not cand_coords: continue

        relative_vec = subtract_vectors(cand_coords, ref_coords_after_swap)
        if not relative_vec: continue
        
        proj_primary = dot_product(relative_vec, view_params['primary_vec'])
        proj_orthogonal = dot_product(relative_vec, view_params['orthogonal_vec'])
        
        if proj_primary > HORIZONTAL_SEPARATION_TOLERANCE and \
           proj_primary > abs(proj_orthogonal):
            count += 1
                
    ref_desc_after_swap = get_object_description(objects_modified, ref_idx_after_swap)
    swap_desc1 = f"{swap_group1_attrs}"
    swap_desc2 = f"{swap_group2_attrs}"
    
    action_taken_desc = "attempting to swap" if num_to_swap > 0 else "no swaps performed (groups empty or no matches)"
    
    return (f"After {action_taken_desc} {swap_desc1} with {swap_desc2}, "
            f"there are {count} objects {query_direction_str} of {ref_desc_after_swap} "
            f"from the {viewpoint_str} viewpoint.")

# --- Main Execution Block ---
if __name__ == "__main__":
    base_path = "/Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/"
    json_files_to_process = ["CLEVR_new_000000.json", "CLEVR_new_000001.json"]

    for filename in json_files_to_process:
        json_file_path = base_path + filename
        print(f"\n\n🚀 Processing file: {json_file_path}")
        print("=" * (len(json_file_path) + 20))
        
        try:
            with open(json_file_path, 'r') as f:
                scene_data = json.load(f)
        except FileNotFoundError:
            print(f"❌ Error: JSON file not found at {json_file_path}")
            continue 
        except json.JSONDecodeError:
            print(f"❌ Error: Could not decode JSON from {json_file_path}")
            continue 
        except Exception as e:
            print(f"❌ An unexpected error occurred loading {json_file_path}: {e}")
            continue 

        ref_attrs_q1, ref_attrs_q2, q2_view, q2_direction = {}, {}, "", ""
        ref_attrs_q2_alt, q2_alt_view, q2_alt_direction = {}, "", ""
        swap_g1_q3, swap_g2_q3, ref_attrs_q3, q3_view, q3_direction = {}, {}, {}, "", ""
        swap_g3_color_q3_alt, swap_g3_shape_q3_alt, ref_attrs_q3_alt, q3_alt_view, q3_alt_direction = {}, {}, {}, "", ""

        if filename == "CLEVR_new_000000.json":
            print(f"📋 Using examples for {filename}")
            ref_attrs_q1 = {"size": "large", "color": "purple", "material": "metal", "shape": "cube"}
            
            ref_attrs_q2 = {"size": "small", "color": "red", "material": "metal", "shape": "cube"}
            q2_view = "right_side"
            q2_direction = "left"
            
            ref_attrs_q2_alt = {"size": "large", "color": "gray", "material": "metal", "shape": "cylinder"}
            q2_alt_view = "front"
            q2_alt_direction = "front"

            swap_g1_q3 = {'color': 'red'}
            swap_g2_q3 = {'shape': 'cube'}
            ref_attrs_q3 = {"size": "large", "color": "gray", "material": "metal", "shape": "cylinder"}
            q3_view = "front"
            q3_direction = "left"

            swap_g3_color_q3_alt = {'color': 'green'}
            swap_g3_shape_q3_alt = {'shape': 'cylinder'}
            ref_attrs_q3_alt = {"size": "small", "color": "red", "material": "metal", "shape": "cube"}
            q3_alt_view = "front"
            q3_alt_direction = "behind"

        elif filename == "CLEVR_new_000001.json":
            print(f"📋 Using examples for {filename}")
            ref_attrs_q1 = {"size": "large", "color": "yellow", "material": "rubber", "shape": "sphere"}

            ref_attrs_q2 = {"size": "small", "color": "blue", "material": "metal", "shape": "cube"}
            q2_view = "left_side"
            q2_direction = "right"
            
            ref_attrs_q2_alt = {"size": "large", "color": "red", "material": "rubber", "shape": "cylinder"}
            q2_alt_view = "front"
            q2_alt_direction = "behind"

            swap_g1_q3 = {'color': 'yellow'}
            swap_g2_q3 = {'shape': 'sphere'}
            ref_attrs_q3 = {"size": "large", "color": "red", "material": "rubber", "shape": "cylinder"}
            q3_view = "right_side"
            q3_direction = "front"
            
            swap_g3_color_q3_alt = {'color': 'brown'}
            swap_g3_shape_q3_alt = {'shape': 'cylinder'}
            ref_attrs_q3_alt = {"size": "small", "color": "brown", "material": "rubber", "shape": "cylinder"}
            q3_alt_view = "front"
            q3_alt_direction = "behind"
        else:
            print(f"🤷 No specific examples defined for {filename}. Skipping detailed tests.")
            continue

        print("\n--- Question Type 1: Nearest Object ---")
        result_q1 = evaluate_q1_nearest_object(scene_data, "front", ref_attrs_q1)
        print(f"Q1 Result: {result_q1}")
        print("-" * 70)

        print("\n--- Question Type 2: Relative Position (Horizontal Only) ---")
        print(f"Q2 Query: View: {q2_view}, Direction: {q2_direction}, Ref: {ref_attrs_q2}")
        result_q2 = evaluate_q2_relative_position(scene_data, q2_view, q2_direction, ref_attrs_q2)
        print(f"Q2 Result:")
        if isinstance(result_q2, list): # Successfully found objects (list might be empty)
            if result_q2:
                for desc in result_q2: print(f"- {desc}")
            else: # Empty list means no objects matched criteria, not an error from the function
                 ref_obj_desc = get_object_description(scene_data.get('objects',[]), find_object_index_by_attrs(scene_data.get('objects',[]), ref_attrs_q2))
                 print(f"No objects found {q2_direction} of {ref_obj_desc} from the {q2_view} viewpoint.")
        else: # Not a list, so it's an error string or specific "no objects found" string from function
            print(result_q2)

        if ref_attrs_q2_alt:
            print(f"\nQ2 Alternate Query: View: {q2_alt_view}, Direction: {q2_alt_direction}, Ref: {ref_attrs_q2_alt}")
            result_q2_alt = evaluate_q2_relative_position(scene_data, q2_alt_view, q2_alt_direction, ref_attrs_q2_alt)
            print(f"Q2 Alternate Result:")
            if isinstance(result_q2_alt, list):
                if result_q2_alt:
                    for desc in result_q2_alt: print(f"- {desc}")
                else:
                    ref_obj_desc = get_object_description(scene_data.get('objects',[]), find_object_index_by_attrs(scene_data.get('objects',[]), ref_attrs_q2_alt))
                    print(f"No objects found {q2_alt_direction} of {ref_obj_desc} from the {q2_alt_view} viewpoint.")
            else:
                print(result_q2_alt)
        print("-" * 70)

        print("\n--- Question Type 3: Swap and Count Relative ---")
        print(f"Q3 Query: Swap {swap_g1_q3} with {swap_g2_q3}. View: {q3_view}, Direction: {q3_direction}, Ref: {ref_attrs_q3}")
        result_q3 = evaluate_q3_swap_and_count_relative(
            scene_data, q3_view, 
            swap_g1_q3, swap_g2_q3,
            q3_direction, ref_attrs_q3
        )
        print(f"Q3 Result: {result_q3}")
        
        if swap_g3_color_q3_alt:
            print(f"\nQ3 Alternate Query: Swap {swap_g3_color_q3_alt} with {swap_g3_shape_q3_alt}. View: {q3_alt_view}, Direction: {q3_alt_direction}, Ref: {ref_attrs_q3_alt}")
            result_q3_alt = evaluate_q3_swap_and_count_relative(
                scene_data, q3_alt_view,
                swap_g3_color_q3_alt, 
                swap_g3_shape_q3_alt,
                q3_alt_direction, ref_attrs_q3_alt 
            )
            print(f"Q3 Alternate Result: {result_q3_alt}")
        print("-" * 70)



🚀 Processing file: /Users/ayush/Documents/GitHub/clevr-dataset-gen/Metadata/output_scenes/CLEVR_new_000000.json
📋 Using examples for CLEVR_new_000000.json

--- Question Type 1: Nearest Object ---
Q1 Result: The nearest object to the large purple metal cube is the small red metal cube (distance: 2.17). (Context: Viewpoint 'front')
----------------------------------------------------------------------

--- Question Type 2: Relative Position (Horizontal Only) ---
Q2 Query: View: right_side, Direction: left, Ref: {'size': 'small', 'color': 'red', 'material': 'metal', 'shape': 'cube'}
Q2 Result:
- the large purple metal cube
- the large gray metal cylinder
- the large blue rubber cylinder
- the small green metal cylinder
- the small gray metal cube

Q2 Alternate Query: View: front, Direction: front, Ref: {'size': 'large', 'color': 'gray', 'material': 'metal', 'shape': 'cylinder'}
Q2 Alternate Result:
- the large blue rubber cylinder
--------------------------------------------------------